In [3]:
import requests
import random
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.auto import tqdm, trange

In [1]:
from LID import LanguageDetector
    
LD = LanguageDetector()

## get pages

In [2]:
base_url = 'http://mokshapr.ru'

In [4]:
downloaded_pages = dict()
urls_queue = {base_url}

In [7]:
import concurrent.futures
import threading


urls_queue_lock = threading.Lock()
downloaded_pages_lock = threading.Lock()


def process_url(url):
    try:
        html = requests.get(url).text
        with downloaded_pages_lock:
            downloaded_pages[url] = html
            if len(downloaded_pages) % 500 == 0:
                print(len(downloaded_pages))


        soup = BeautifulSoup(html, "html.parser")

        new_urls = set()
        for a in soup.findAll('a'):
            if not a.has_attr('href'):
                continue
            href = a['href']
            if href.startswith('#'):
                continue
            if not href.startswith('/') and not 'mokshapr.ru' in href:
                continue

            if href.startswith('/'):
                new_url = base_url + href
            else:
                new_url = href

            with downloaded_pages_lock:
                if new_url not in downloaded_pages:
                    new_urls.add(new_url)

        return new_urls
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return set()

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    while True:
        futures = []
        urls_to_process = []

        with urls_queue_lock:
            if not urls_queue:
                break
            for _ in range(min(10, len(urls_queue))):
                urls_to_process.append(urls_queue.pop())

        for url in urls_to_process:
            futures.append(executor.submit(process_url, url))

        for future in concurrent.futures.as_completed(futures):
            new_urls = future.result()
            with urls_queue_lock:
                urls_queue.update(new_urls)


1000
Error processing https://mokshapr.ru/politika/trudoustrojstvo-uchastnikov-svo-proforientacziya-podrostkov-perepodgotovka-kadrov-partproekt-moya-karera-s-edinoj-rossiej-predstavil-itogi-raboty-s-nachala-goda/: HTTPSConnectionPool(host='mokshapr.ru', port=443): Max retries exceeded with url: /politika/trudoustrojstvo-uchastnikov-svo-proforientacziya-podrostkov-perepodgotovka-kadrov-partproekt-moya-karera-s-edinoj-rossiej-predstavil-itogi-raboty-s-nachala-goda/ (Caused by SSLError(SSLError(1, '[SSL] record layer failure (_ssl.c:2580)')))
1500
2000
Error processing https://mokshapr.ru/politika/trudoustrojstvo-zhenshhin-stanet-otdelnym-napravleniem-partproekta-moya-karera-s-edinoj-rossiej/: HTTPSConnectionPool(host='mokshapr.ru', port=443): Max retries exceeded with url: /politika/trudoustrojstvo-zhenshhin-stanet-otdelnym-napravleniem-partproekta-moya-karera-s-edinoj-rossiej/ (Caused by SSLError(SSLError(1, '[SSL] record layer failure (_ssl.c:2580)')))
Error processing https://mokshapr

## parse pages

In [20]:
parsed_pages = []

for url in tqdm(downloaded_pages):
    if '?start=' in url:
        continue
    html = downloaded_pages[url]
    soup = BeautifulSoup(html)
    try:
        title = soup.find("h1", class_="tdb-title-text").text.strip()
        category = soup.find("div", class_="tdb_single_categories").find("a").text.strip()
        body = "\n".join(p.text.strip() for p in soup.find("div", class_="tdb_single_content").find_all("p"))
    except AttributeError as e:
        print(st)
        continue
    parsed_pages.append({
        'url': url,
        'title': title, 
        'category': category,
        'body': body,
        'mdf_score': LD.predict_lang(body, k=10)['mdf']
    })

  0%|          | 0/5178 [00:00<?, ?it/s]

category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
category
c

In [21]:
print(len(parsed_pages))

3872


## get df with data

In [22]:
df = pd.DataFrame(parsed_pages)

In [38]:
df[df.mdf_score>0.5].shape

(1748, 5)

In [40]:
mdf_news = df[df.mdf_score>0.5][['url', 'title', 'category', 'body']]

In [41]:
mdf_news

,url,title,category,body
0,https://mokshapr.ru/obo-vsem-ponemnogu/mzyara-...,Мзяра питнес стяй шабать школав анокламац,Обо всем понемногу,Тонафнемань од кизоти ляткшни недяля. Мзярс ша...
1,https://mokshapr.ru/novosti/kultura/zhuvatan-p...,Жуватань пчкафтомати макссь нильгемонь киза!,Культура,Мокшавась Нина Куркина—Рузаевскяй райононь Мо...
2,https://mokshapr.ru/obo-vsem-ponemnogu/%d1%8d%...,Эрь пожарнайсь—герой,Обо всем понемногу,"Лама веконь историяста российскяй ошне, велетн..."
3,https://mokshapr.ru/novosti/sport/pyalkstomatn...,Пялькстоматне лемтьфтольхть спецоперациянь гер...,Спорт,"21 декабрьста Инсар ошса, В.Я.Антроповонь лемс..."
4,https://mokshapr.ru/novosti/kultura/mokshen-ky...,Мокшень кяльти кельгомась ётась тядянзон-алянз...,Культура,Идень садса воспитателькс работамась—тевсь аф ...
...,...,...,...,...
3852,https://mokshapr.ru/nacproekty/minczonk-petevi...,Миньцонк петевихть китне,Нацпроекты,«Безопасные качественные дороги» нацпроектть э...
3853,https://mokshapr.ru/nacproekty/id-marhta-semya...,Идь мархта семьятненди лездыхть ипотекать панд...,Нацпроекты,«Финансовая поддержка семей при рождении детей...
3859,https://mokshapr.ru/nacproekty/viiyaftovi-medi...,Виияфтови медицинскяй услугань экспортсь,Нацпроекты,Мордовияса мольфтеви «Здравоохранение» нацпрое...
3868,https://mokshapr.ru/novosti/kultura/mazyht-mok...,Мазыхть мокшень стиреньке,Культура,Кельгома лувиеньке! Газетасонк сяда тов мольфт...


In [43]:
mdf_news.to_csv('results/moksha_pravda.tsv', sep='\t', index=None)